# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.



In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import gmaps.datasets
import json

# Import API key
from api_keys import g_key



### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
city_data_df = pd.read_csv("weather_data.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Date,Latitude,Longitude,Max temperature,Humidity,Cloudiness,Wind Speed
0,0,Serang,ID,1638054421,-6.1149,106.1502,74.97,87,100,7.43
1,1,Atuona,PF,1638054408,-9.8000,-139.0333,78.84,72,37,19.89
2,2,Hermanus,ZA,1638054323,-34.4187,19.2345,64.40,81,31,17.36
3,3,Hobart,AU,1638054384,-42.8794,147.3294,62.46,55,20,9.22
4,4,Port Keats,AU,1638054422,-14.2500,129.5500,86.05,53,100,3.71


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=g_key)


In [5]:
#Use the Lat and Lng as locations and Humidity as the weight.
locations = city_data_df[['Latitude', 'Longitude']]
weight = city_data_df['Humidity']

# Plot heatmap
fig = gmaps.figure()

# Add a heat layer to the map
heat_layer = gmaps.heatmap_layer(locations, weights=weight, 
                                 dissipating=False, max_intensity=100, point_radius=3)

# Add the heat layer
fig.add_layer(heat_layer)

# Show the heat map
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# Narrow down the DataFrame to find your ideal weather condition.
filtered_weather_df = city_data_df
narrowed_city_df = filtered_weather_df.loc[(filtered_weather_df["Max temperature"] > 70) & (filtered_weather_df["Max temperature"] < 80) & (filtered_weather_df["Cloudiness"] == 0), :]
narrowed_city_df = narrowed_city_df.dropna(how='any')
narrowed_city_df.reset_index(inplace=True)
del narrowed_city_df['index']
del narrowed_city_df['Unnamed: 0']
narrowed_city_df

,City,Country,Date,Latitude,Longitude,Max temperature,Humidity,Cloudiness,Wind Speed
0,Umluj,SA,1638054462,25.0213,37.2685,79.02,53,0,8.95
1,Salalah,OM,1638054469,17.0151,54.0924,73.49,46,0,3.44
2,Yulara,AU,1638054405,-25.2406,130.9889,73.24,27,0,16.11
3,Vila Velha,BR,1638054559,-20.3297,-40.2925,75.15,83,0,9.22
4,Guerrero Negro,MX,1638054570,27.9769,-114.0611,73.38,46,0,13.78
5,Mandurah,AU,1638054572,-32.5500,115.7000,70.05,48,0,3.33
6,Diego de Almagro,CL,1638054640,-26.3667,-70.0500,71.49,24,0,8.52
7,Ures,MX,1638054666,29.4333,-110.4000,75.33,31,0,3.83
8,Pimentel,PE,1638054690,-6.8367,-79.9342,70.09,73,0,21.85


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = narrowed_city_df

#params dictionary to update each iteration

params = {"radius": 5000, 
         "types": "lodging",
         "key": g_key}

for index,row in hotel_df.iterrows():
    lat = row['Latitude']
    lon = row['Longitude']
    
    #change location each iteration while leaving original paras in place
    
    params['location'] = f"{lat},{lon}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    #assemble url and make API request
    
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url,params=params).json()
    
    #print(json.dumps(response, indent=4, sort_keys=True))
    
    results = response['results']
 
    
    try:
        
        print(f"Closest hotel is {results[0]['name']}.")
        #Add Hotel name to the DataFrame
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']

    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
    print("---------")

Retrieving Results for Index 0: Umluj.
Closest hotel is Moon light Furnished Units.
---------
Retrieving Results for Index 1: Salalah.
Closest hotel is HAMDAN PLAZA HOTEL SALALAH, an HTG Hotel.
---------
Retrieving Results for Index 2: Yulara.
Closest hotel is Desert Gardens Hotel - Ayers Rock Resort.
---------
Retrieving Results for Index 3: Vila Velha.
Closest hotel is Quality Suites Vila Velha.
---------
Retrieving Results for Index 4: Guerrero Negro.
Closest hotel is Casa Laguna, Bed & Breakfast.
---------
Retrieving Results for Index 5: Mandurah.
Closest hotel is Atrium Hotel Mandurah.
---------
Retrieving Results for Index 6: Diego de Almagro.
Closest hotel is hotel luna del desierto.
---------
Retrieving Results for Index 7: Ures.
Closest hotel is Hotel HACIENDA San Francisco.
---------
Retrieving Results for Index 8: Pimentel.
Closest hotel is Casona Del Muelle.
---------


In [8]:
hotel_df

,City,Country,Date,Latitude,Longitude,Max temperature,Humidity,Cloudiness,Wind Speed,Hotel Name
0,Umluj,SA,1638054462,25.0213,37.2685,79.02,53,0,8.95,Moon light Furnished Units
1,Salalah,OM,1638054469,17.0151,54.0924,73.49,46,0,3.44,"HAMDAN PLAZA HOTEL SALALAH, an HTG Hotel"
2,Yulara,AU,1638054405,-25.2406,130.9889,73.24,27,0,16.11,Desert Gardens Hotel - Ayers Rock Resort
3,Vila Velha,BR,1638054559,-20.3297,-40.2925,75.15,83,0,9.22,Quality Suites Vila Velha
4,Guerrero Negro,MX,1638054570,27.9769,-114.0611,73.38,46,0,13.78,"Casa Laguna, Bed & Breakfast"
5,Mandurah,AU,1638054572,-32.5500,115.7000,70.05,48,0,3.33,Atrium Hotel Mandurah
6,Diego de Almagro,CL,1638054640,-26.3667,-70.0500,71.49,24,0,8.52,hotel luna del desierto
7,Ures,MX,1638054666,29.4333,-110.4000,75.33,31,0,3.83,Hotel HACIENDA San Francisco
8,Pimentel,PE,1638054690,-6.8367,-79.9342,70.09,73,0,21.85,Casona Del Muelle


In [9]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]




In [11]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))